In [ ]:
import psycopg2
conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

def get_data(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
    return data

def get_colname(table):
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM {} LIMIT 1;'.format(table))
        data = [col.name for col in cur.description]
    return data

def exec_transaction(sqls):
    with conn.cursor() as cur:
        for sql in sqls:
            cur.execute(sql)
    conn.commit()

In [ ]:
import mysql.connector
conn2 = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'honomara',
    password =  'honomara',
    database = 'honomara',
)

cur2 = conn2.cursor(dictionary=True)

In [ ]:
create_members_table = '''
CREATE TABLE IF NOT EXISTS members (
    `member_id` INT PRIMARY KEY AUTO_INCREMENT,
    `family_name` VARCHAR(20) NOT NULL,
    `first_name` VARCHAR(20) NOT NULL,
    `show_name` VARCHAR(20) NOT NULL, -- used for after and results
    `kana` VARCHAR(40) NOT NULL, -- for index
    `year` INT NOT NULL,
    `sex` INT NOT NULL DEFAULT 0, -- 0=man, 1=woman
    `visible` boolean NOT NULL DEFAULT true
);
'''

cur2.execute(create_members_table)
conn2.commit()

In [ ]:
import re
import MeCab

def name_len(name):
    l=len(name)
    l-=name.count(')')
    l-=name.count('(')
    l-=name.count('）')
    l-=name.count('（')
    return l
colname = get_colname('person')
mecab = MeCab.Tagger("-Ochasen")
regkana=re.compile(r'[ァ-ヶー]+') 

for fields in get_data('SELECT * FROM person ORDER BY class;'):
    item = dict(zip(colname,fields))
    if not item['name']:
        name = item['fullname']
    elif not item['fullname']:
        name = item['name']
    elif name_len(item['fullname']) > name_len(item['name']):
        name = item['fullname']
    else:
        name = item['name']
    fam_n = re.match("^[^（()）]+" , item['after_name']).group()
    
    fir_n = re.match("[^（()）　]*$" , name.replace(fam_n,''))
    if not fir_n or len(fir_n.group()) == 0:
        fir_n = '不明'
        parse_name = fam_n
    else:
        fir_n = fir_n.group()
        fir_n = fir_n.replace(' ','')
        fir_n = fir_n.replace('　','')
        parse_name = fam_n + fir_n
    if re.match(".*[（()）].*" , name):
        after_name = name
    else:
        after_name = item['after_name']
    parse_result = mecab.parse(parse_name)#ふりがなを取得
    kana_list = regkana.findall(parse_result.replace('\n',' '))  #その行のカタカナを取り出す(list)
    kana = ''.join(kana_list)
    year = item['class'] + 2000 - 9
    sex = '男' if item['sex'] == 0 else '女'  
    cur2.execute('''
    INSERT INTO members (member_id,family_name,first_name,show_name,kana,year,sex,visible)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s);''',(item['person_id'],fam_n,fir_n,after_name,kana,year,item['sex'],item['visible']))
conn2.commit()

In [ ]:
import csv


# replace the path to the csv!!!!!!!!
csv_path = "replace here !!!!!!!!"
# in most case this path would be /vagrant/member.csv or like that

with open(csv_path,"r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        cur2.execute('''UPDATE members SET kana = '{}' WHERE member_id = {};'''.format(row['kana'], int(row['person_id'])))
conn2.commit()